# Setting Up Work Environment

In [8]:
!pip install --upgrade google-generativeai

In [9]:
!pip install -q -U google-genai

In [10]:
import os
from google import genai
import google.generativeai as ggenai
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

from PIL import Image
from google.genai import types

from IPython.display import HTML

In [11]:
# Set up the API key (Replace 'YOUR_API_KEY' with your actual Gemini API key)
key = userdata.get('genai_api')
client = genai.Client(api_key=key)

List the set of available models

In [12]:
ggenai.configure(api_key=key)

models = ggenai.list_models()
for model in models:
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
model

Create a helper function to make it easier to use prompts and look at generated outputs.

In [13]:
def get_completion(prompt, model="gemini-1.5-flash"):
  response = client.models.generate_content(
      model=model,
      contents=prompt,
      config=types.GenerateContentConfig(max_output_tokens=100, temperature=0.1)
      )
  return response.text  # Extract the generated text

# Sentiment Analysis of Product Review

Sentiment analysis is a natural language processing (NLP) technique used to identify and interpret the sentiment expressed in text. It is particularly valuable for analyzing product reviews, enabling businesses to gain insights into customer opinions and feedback. In this example, we’ll apply sentiment analysis to a product review specifically, a review of a lamp. Here’s the review:

In [14]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [15]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is overwhelmingly **positive**.  The reviewer praises the lamp's features (storage, price), the fast shipping, the company's excellent customer service in resolving the broken string and missing part issues, and expresses overall satisfaction with both the product and the company (Lumina).



In [16]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positive



# Identify types of emotions

Let’s explore another use case where prompt engineering can be applied to perform a more advanced sentiment analysis task emotion detection. Large language models are highly effective at extracting specific information from text, including identifying the emotions expressed. This can be especially useful for understanding how customers feel about a product. For customer support teams, in particular, detecting whether a user is extremely upset can be critical for prioritizing responses.

Below is a sample prompt designed to extract emotional cues from customer feedback:

In [17]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfaction, gratitude, relief, happiness, trust



# Identify Anger

In some cases, your classification task may focus on detecting a specific emotion such as identifying whether a customer is angry. Knowing when a customer expresses strong negative emotions can be critical. For instance, if someone is clearly angry, it may warrant special attention from customer support or customer success teams, who can reach out proactively to understand the issue and work toward a resolution.

In [18]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

no



# Extract Product & Company Names From Customer Reviews

Information extraction is a key task in natural language processing (NLP) that involves identifying and retrieving specific pieces of information from a given text. In the following prompt, we’ll ask the language model to extract two specific details: the item that was purchased and the name of the company that manufactured it.

In [22]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible. \

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Item": "lamp",
  "Brand": "Lumina"
}
```



# Doing Multiple Tasks at Once

In the examples we've explored, we created prompts to detect sentiment, determine if a reviewer is angry, and extract both the purchased item and its brand.

One approach to extract all this information is to use multiple prompts calling the `get_completion` function three or four times, once for each piece of information. However, a more efficient method is to combine everything into a single prompt that captures all the desired outputs at once.

For example, you can instruct the model to:
"Identify the following: sentiment, whether the reviewer is expressing anger, the item purchased, and the company that made it."
You can also specify the format—for instance, requesting the anger value as a Boolean (`true` or `false`).

In [23]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```



# Topics Inferring

Another advanced NLP task you can perform is topic inference determining the main themes or subjects within a piece of text. Given a long article or document, the goal is to understand what it's about and what topics are being discussed.

For example, consider this fictitious newspaper article summarizing a recent government survey about how agency employees feel about their workplace.

In [24]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

In [25]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Employee satisfaction, NASA, Government survey, Job satisfaction, Social Security



In [27]:
response.split(sep=',')

['Employee satisfaction',
 ' NASA',
 ' Government survey',
 ' Job satisfaction',
 ' Social Security\n']

# Make Alert for Certain Topics

If you have a collection of articles and extract topics from each, you can leverage a large language model to help organize and index the content by topic. This makes it easier to navigate and retrieve information based on specific themes. For this example, let’s work with a slightly different set of topics to illustrate how this can be done.

In [26]:
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

In [31]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1



In [33]:
# Check that each item can be split into at least two parts before creating the dictionary
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n') if len(i.split(': ')) >= 2}
if 'nasa' in topic_dict and topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [36]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1 and put all the results in a dictionary format

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "nasa": 1,
  "local government": 0,
  "engineering": 0,
  "employee satisfaction": 1,
  "federal government": 1
}
```

